In [1]:
import os
from pathlib import Path


random_graphs_path = Path("/home/timgarrels/Projects/masterthesis/datasets/random_graphs")
random_graphs = os.listdir(random_graphs_path)

In [2]:
import time
import networkx as nx

def time_avg_cc(g: nx.Graph) -> float:
    start = time.time()
    avg_cc = nx.average_clustering(g)
    return time.time() - start

In [3]:
from typing import List


def exclusive_neighbors(g: nx.Graph, node: int, node_set: List[int]) -> List[int]:
    neighbors = g.neighbors(node)
    
    exclusive_neighbors = []

    for neighbor in neighbors:
        exclusive = True
        for n in node_set:
            if g.has_edge(neighbor, n):
                exclusive = False
                break
        if exclusive:
            exclusive_neighbors.append(neighbor)
    return exclusive_neighbors


In [4]:
def esu(g: nx.Graph, k: int=4):
    subgraphs = []
    def extend_subgraph(v_subgraph: List[int], v_extension: List[int], node: int):
        if len(v_subgraph) == k:
            subgraphs.append(v_subgraph)
            return
        
        while len(v_extension) > 0:
            w = v_extension.pop()
            new_v_extension = v_extension + exclusive_neighbors(g, w, v_subgraph)
            extend_subgraph(v_subgraph + [w], new_v_extension, node)
        
    for node in g.nodes:
        v_extension = [neighbor for neighbor in g.neighbors(node) if neighbor > node]
        extend_subgraph([node], v_extension, node)
    return subgraphs
    

In [5]:
def time_esu(g: nx.Graph, k: int=4) -> float:
    start = time.time()
    subgraphs = esu(g, k)
    return time.time() - start

In [6]:
from tqdm import tqdm
import pandas as pd

OUT = "esu_timings.csv"

if not Path(OUT).is_file():
    iterations = 5
    pbar = tqdm(total=len(random_graphs) * iterations)

    rows = []
    for r in random_graphs:
        g = nx.read_edgelist(random_graphs_path / r)
        for i in range(iterations):
            rows.append({
                "graph": r,
                "iteration": i,
                "esu_time": time_esu(g),
                "avg_cc_time": time_avg_cc(g),
            })
            pbar.update(1)
    
    pd.DataFrame(rows).to_csv(OUT)

df = pd.read_csv(OUT)

In [7]:
def get_graph_type(graph: str):
    number, *parts = graph.split("_")
    return "_".join(parts)

df["graph_type"] = df["graph"].apply(get_graph_type)

In [8]:
df

,Unnamed: 0,graph,iteration,esu_time,avg_cc_time,graph_type
0,0,3_barabasi_albert_graph_m_3,0,21.885886,0.016100,barabasi_albert_graph_m_3
1,1,3_barabasi_albert_graph_m_3,1,22.124892,0.015252,barabasi_albert_graph_m_3
2,2,3_barabasi_albert_graph_m_3,2,22.115742,0.015070,barabasi_albert_graph_m_3
3,3,3_barabasi_albert_graph_m_3,3,22.002315,0.014956,barabasi_albert_graph_m_3
4,4,3_barabasi_albert_graph_m_3,4,21.896368,0.014927,barabasi_albert_graph_m_3
...,...,...,...,...,...,...
170,170,0_barabasi_albert_graph_m_3,0,11.433270,0.014571,barabasi_albert_graph_m_3
171,171,0_barabasi_albert_graph_m_3,1,11.478666,0.014481,barabasi_albert_graph_m_3
172,172,0_barabasi_albert_graph_m_3,2,11.606124,0.014201,barabasi_albert_graph_m_3
173,173,0_barabasi_albert_graph_m_3,3,11.612031,0.014307,barabasi_albert_graph_m_3


In [9]:
from statistics import mean

def mean_group(l):
    try:
        return mean(l)
    except TypeError:
        return list(l)[0]

df = df.groupby("graph").agg(mean_group)

df["factor"] = df["esu_time"] / df["avg_cc_time"]
df = df.sort_values(by="graph_type")
df

,Unnamed: 0,iteration,esu_time,avg_cc_time,graph_type,factor
graph,,,,,,
0_barabasi_albert_graph_m_1,62,2,0.350095,0.005334,barabasi_albert_graph_m_1,65.631741
2_barabasi_albert_graph_m_1,17,2,0.681487,0.005609,barabasi_albert_graph_m_1,121.503303
3_barabasi_albert_graph_m_1,52,2,0.805222,0.005691,barabasi_albert_graph_m_1,141.483256
1_barabasi_albert_graph_m_1,97,2,0.669867,0.005595,barabasi_albert_graph_m_1,119.734758
4_barabasi_albert_graph_m_1,152,2,0.307093,0.005363,barabasi_albert_graph_m_1,57.260616
3_barabasi_albert_graph_m_2,92,2,5.000407,0.009485,barabasi_albert_graph_m_2,527.172158
2_barabasi_albert_graph_m_2,162,2,4.900376,0.009524,barabasi_albert_graph_m_2,514.549125
1_barabasi_albert_graph_m_2,142,2,5.193014,0.009619,barabasi_albert_graph_m_2,539.862572
0_barabasi_albert_graph_m_2,112,2,3.091611,0.009678,barabasi_albert_graph_m_2,319.462018


In [10]:
def extract_number(graph_name: str):
    return graph_name.split("_")[0]

In [11]:
label_lookup = {
    'barabasi_albert_graph_m_1': r"\textsc{Barabási-Albert} $m_1$",
    'barabasi_albert_graph_m_2': r"\textsc{Barabási-Albert} $m_2$",
    'barabasi_albert_graph_m_3': r"\textsc{Barabási-Albert} $m_3$",
    'ferdos_renyi_graph_m_2000': r"\textsc{Erdős–Rényi}",
    'scale_free_graph_a_28_b_7_g_02': r"\textsc{scale-free\textsubscript{28}",
    'scale_free_graph_a_35_b_3_g_35': r"\textsc{scale-free\textsubscript{35}",
    'scale_free_graph_a_65_b_1_g_25': r"\textsc{scale-free\textsubscript{25}",
}

In [12]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.lines as mlines

plt.rcParams.update({'font.size': 18})
plt.rcParams['text.usetex'] = True


graph_types = sorted(set(df["graph_type"]))

colors_for_graph_types = dict(zip(
    set(df["graph_type"]),
    list(mcolors.TABLEAU_COLORS.keys())[:len(set(df["graph_type"]))]
))


fig, ax = plt.subplots(1,1, figsize=(8,7), dpi=1024)

# Data Plotting
for gt in graph_types:
    gt_df = df[df["graph_type"] == gt].copy()
    gt_df = gt_df[gt_df["factor"] == gt_df["factor"].min() ]
    
    gt_df = gt_df.sort_values("graph_type")
    
    ax.scatter(
        gt_df.index,
        gt_df["avg_cc_time"],
        label="avg_cc_time",
        color="tab:orange",
        # color=colors_for_graph_types[gt],
    )
    
    ax.scatter(
        gt_df.index,
        gt_df["esu_time"],
        label="esu_time",
        marker="x",
        color="tab:blue",
        # color=colors_for_graph_types[gt],
    )

# Axis
ax.set_ylabel("Runtime (s)")
ax.set_xlabel("Graph Instance")

ax.set_yscale("log")

ax.set_xticklabels(list(map(lambda x: label_lookup[x], graph_types)), rotation=45, ha="right")
# Legend


labels = ["Runtime ESU", "Runtime Avg. CC"]

handles = [
    mlines.Line2D(
        [], [],
        color="tab:blue",
        marker='x',
        linestyle='None',
        markersize=8,
        label="",
    ),
    mlines.Line2D(
        [], [],
        color="tab:orange",
        marker='o',
        linestyle='None',
        markersize=8,
        label="",
    )
]

ax.legend(labels=labels, handles=handles)
fig.tight_layout()

/tmp/ipykernel_5865/222799237.py:49: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(list(map(lambda x: label_lookup[x], graph_types)), rotation=45, ha="right")


In [13]:
fig.savefig("esu_runtimes.pdf")
fig.savefig("esu_runtimes.png")